In [1]:
from collections import defaultdict, namedtuple, Counter
import pickle
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import pyranges as pr
import pysam
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from scipy.stats import variation, gaussian_kde, pearsonr, mannwhitneyu

# intron retention

## nanopore

In [8]:
IR_stats = namedtuple('IR_stats', 'intron_total_count intron_count unsplice_count unsplice_intron')

splice_stats_pkl = '/public/home/mowp/data/total_RNA/nanopore_cdna/polyadenylated_data/nanopore_cdna.full_len.bed.ir_stats.tsv'

with open(splice_stats_pkl+'.pkl', 'rb') as f:
    ir_stats_dict = pickle.load(f)

In [9]:
ir_stats_dict['dfce655d-887f-47f2-8bc2-89767009db20']

{'AT1G01100': IR_stats(intron_total_count=3, intron_count=3, unsplice_count=0, unsplice_intron='')}

In [18]:
infile = '/public/home/mowp/data/total_RNA/nanopore_cdna/polyadenylated_data/nanopore_cdna.full_len.bam'
ir_isoform = Counter()
with pysam.AlignmentFile(infile, 'rb') as inbam:
    for read in inbam:
        gene_id = read.get_tag('gi')
        try:
            intron_total_count, intron_count, unspliced_count, unspliced_intron  = ir_stats_dict[read.query_name][gene_id]
            if intron_count == 0:
                # 转录还没有过第一个exon的情况
                # 忽略
                continue
        except KeyError:
            continue
        
        if unspliced_count > 0:
            ir_isoform[(gene_id, intron_count, unspliced_count, unspliced_intron)] += 1

In [22]:
ir_isoform_results = []
for k, v in ir_isoform.items():
    ir_isoform_results.append((*k, v))

In [24]:
df = pd.DataFrame(ir_isoform_results, columns=('gene_id', 'total_intron_counts', 'retention_intron_counts', 'retention_intron_number', 'isoform_read_counts'))

In [26]:
df.to_csv('/public/home/mowp/data/total_RNA/notebook/splicing_stats/nanopore_ir_stats.csv', index=False)

## pacbio

In [27]:
IR_stats = namedtuple('IR_stats', 'intron_total_count intron_count unsplice_count unsplice_intron')

splice_stats_pkl = '/public/home/mowp/data/total_RNA/pacbio_cdna/polyadenylated_data/totalRNA.full_len.bed.ir_stats.tsv'

with open(splice_stats_pkl+'.pkl', 'rb') as f:
    ir_stats_dict = pickle.load(f)

In [28]:
infile = '/public/home/mowp/data/total_RNA/pacbio_cdna/polyadenylated_data/totalRNA.full_len.bam'
ir_isoform = Counter()
with pysam.AlignmentFile(infile, 'rb') as inbam:
    for read in inbam:
        gene_id = read.get_tag('gi')
        try:
            intron_total_count, intron_count, unspliced_count, unspliced_intron  = ir_stats_dict[read.query_name][gene_id]
            if intron_count == 0:
                # 转录还没有过第一个exon的情况
                # 忽略
                continue
        except KeyError:
            continue
        
        if unspliced_count > 0:
            ir_isoform[(gene_id, intron_count, unspliced_count, unspliced_intron)] += 1

In [29]:
ir_isoform_results = []
for k, v in ir_isoform.items():
    ir_isoform_results.append((*k, v))

In [30]:
df = pd.DataFrame(ir_isoform_results, columns=('gene_id', 'total_intron_counts', 'retention_intron_counts', 'retention_intron_number', 'isoform_read_counts'))

In [31]:
df

,gene_id,total_intron_counts,retention_intron_counts,retention_intron_number,isoform_read_counts
0,AT1G01020,8,2,"1,2",1
1,AT1G01020,8,1,4,3
2,AT1G01020,8,1,5,2
3,AT1G01020,8,2,"3,4",3
4,AT1G01020,8,1,2,2
...,...,...,...,...,...
24087,AT5G67600,2,1,2,6
24088,AT5G67610,9,1,3,10
24089,AT5G67640,3,1,2,3
24090,ATMG00580,2,1,1,1


In [32]:
df.to_csv('/public/home/mowp/data/total_RNA/notebook/splicing_stats/pacbio_ir_stats.csv', index=False)

# exon skipping

In [33]:
ES_stats = namedtuple('ES_stats', 'exon_total_count exon_count skipped_exon_count skipped_exon')

es_stats_pkl = '/public/home/mowp/data/total_RNA/nanopore_cdna/polyadenylated_data/nanopore_cdna.full_len.bed.es_stats.tsv.pkl'

with open(es_stats_pkl, 'rb') as f:
    es_stats_dict = pickle.load(f)

In [35]:
es_stats_dict['ad1aa3af-d813-4c7b-82ae-338290ef25b9']

{'AT1G01100': ES_stats(exon_total_count=4, exon_count=4, skipped_exon_count=1, skipped_exon='1')}

In [55]:
infile = '/public/home/mowp/data/total_RNA/nanopore_cdna/polyadenylated_data/nanopore_cdna.full_len.bam'
es_isoform = Counter()
with pysam.AlignmentFile(infile, 'rb') as inbam:
    for read in inbam:
        gene_id = read.get_tag('gi')
        try:
            exon_total_count, exon_count, skipped_exon_count, skipped_exon = es_stats_dict[read.query_name][gene_id]
        except KeyError:
            continue
        
        if skipped_exon_count > 0:
            es_isoform[(gene_id, exon_total_count, skipped_exon_count, skipped_exon)] += 1

In [57]:
es_isoform_results = []
for k, v in es_isoform.items():
    es_isoform_results.append((*k, v))

In [58]:
df = pd.DataFrame(es_isoform_results, columns=('gene_id', 'total_exon_counts', 'skipped_exon_count', 'skipped_exon', 'isoform_read_counts'))

In [59]:
df

,gene_id,total_exon_counts,skipped_exon_count,skipped_exon,isoform_read_counts
0,AT1G01020,9,1,4,2
1,AT1G01020,9,4,"2,3,4,5",1
2,AT1G01050,9,5,"3,4,5,6,7",1
3,AT1G01060,9,1,2,1
4,AT1G01090,3,1,2,1
...,...,...,...,...,...
3669,AT5G67140,5,1,3,3
3670,AT5G67480,4,1,1,1
3671,AT5G67560,6,1,3,2
3672,AT5G67590,5,1,4,7


In [60]:
df.to_csv('/public/home/mowp/data/total_RNA/notebook/splicing_stats/nanopore_es_stats.csv', index=False)